# Simulation of Intense Beams in Synchrotrons

### ... a story of resonances and instabilities.

<br /><br />

##### a notebook talk by [Adrian Oeftiger $\nearrow$](http://oeftiger.net/)

in the [GSI Accelerator Seminar on 3 September 2020 $\nearrow$](https://indico.gsi.de/event/10059/). See also [rendered talk on github $\nearrow$](https://aoeftiger.github.io/GSI-acc-seminar-09-2020/) and [source in github repo $\nearrow$](https://github.com/aoeftiger/GSI-acc-seminar-09-2020/).

<br />
<center>
    <small>(hit space to go to the next slide)</small>
</center>

# Goals of this talk

- showcase new simulation suite for SIS18 and SIS100

- no theory, rather hands-on experience

- practical modelling examples with live simulations

- give overview of applications

# Abstract

<small><p>
This introduction on collective beam dynamics focuses on high-intensity effects
in synchrotrons like SIS18 or SIS100.
The charged particles in the beam interact
(a.) with the beam self-fields they produce and
(b.) with the induced currents in the vacuum pipe.
This interaction leads to corresponding collective effects, 
such as "space charge" (self-field interaction) or 
"beam coupling impedance" (interaction with the surroundings). 
The beam can become resonantly excited or is even driven into 
exponential instabilities over many revolutions in the circular accelerator. 
These effects potentially limit the performance, they scale with the 
number of particles in the beam: their understanding (and mitigation) 
is crucial to safely operate synchrotrons like SIS100 at highest beam intensities.
</p><p>
We dive into the world of these collective effects in beam physics, 
exploring their mechanisms and how they can lead to beam loss. 
During the talk, we also look behind the scenes on how to 
numerically model such long-term effects, in particular employing 
high-performance techniques such as GPU computing.
</p></small>

# How to run this notebook at GSI:

Connect to the GSI high-performance computing cluster `virgo.hpc` (with port forwarding), load the accelerator physics environment and run the jupyter notebook server:

```bash
$ ssh virgo.hpc -L 8888:localhost:8888
$ singularity exec --bind /cvmfs \
    /cvmfs/vae.gsi.de/centos7/containers/user_container-develop.sif \
    bash -l
Singularity> cd /cvmfs/aph.gsi.de/
Singularity> source modules.sh
Singularity> module load aph_all
Singularity> cd
Singularity> git clone https://github.com/aoeftiger/GSI-acc-seminar-09-2020
Singularity> cd GSI-acc-seminar-09-2020
Singularity> jupyter notebook --no-browser --port=8888
```

... and you're ready to open your local browser and type in `localhost:8888`, where you open this notebook.

## How to run this notebook on your individual workstation:

Install [`PyHEADTAIL` $\nearrow$](https://pypi.org/project/PyHEADTAIL/), [`cpymad` $\nearrow$](https://pypi.org/project/cpymad/), [`PySixTrack` $\nearrow$](https://github.com/SixTrack/pysixtrack/) and [`SixTrackLib` $\nearrow$](https://github.com/SixTrack/sixtracklib) in your `python3` environment and run this `jupyter notebook`.

_Note_: if you don't have `openCL` drivers installed, replace all `device=...` instructions in SixTrackLib within this notebook by `device=None` to run on a normal single CPU thread.

# Simulation of Intense Beams in Synchrotrons

### ... a story of resonances and instabilities.

# Structure

1. `PyHEADTAIL`: collective beam dynamics
2. Wakefields &amp; 1st Order Coherent Instabilities
3. `SixTrackLib`: symplectic non-linear 6D tracking
4. Space Charge &amp; Resonances and 2nd Order Coherent Instabilities

# Beam dynamics

3D particle motion $\leadsto$ 6 phase space coordinates:
$$\mathbb{X}=(\underbrace{x, x'\vphantom{y'}}_{horizontal}, \underbrace{y, y'}_{vertical}, \underbrace{z, \delta\vphantom{y'}}_{longitudinal})$$

A beam $=$ state of $N$ macro-particles $=$ $6N$ values of phase space coordinates

## Simulations:
* typically up to $\mathcal{O}(10^6)$ macro-particles
* accelerator elements to track through: up to $\mathcal{O}(1000)$
* simulations can last up to $\mathcal{O}(10^6)$ turns 
* particle-to-particle interaction: binning, FFT, convolution, particle-in-cell, Poisson solvers

# Requirements for simulation tools

* **accurate**: long-term evolution $\leadsto$ double precision
* **fast**: heavy number crunching $\leadsto$ high-performance computing (HPC)<br />(multi-core CPU, multi-nodes (MPI) or GPU parallelisation, in particular for _collective effects_)
* **flexible**: iterative development of accelerator models with frequent updates $\leadsto$ python


<center>
<img src="images/python.png" alt="python logo" style="width:30%;"/>
</center>

# The macro-particle simulation model

**single-particle physics** $\implies$ <span style="color:#00aad4;">"tracking"</span>: particle motion due to external focusing (magnets and RF cavities)

**multi-particle physics** $\implies$ <span style="color:#aa0000;">"collective effect kicks"</span>: direct and indirect particle-to-particle interaction

<center>
<img src="images/transverse-model_empty.png" alt="sketch of one-turn map for ring" style="width:30%;"/>
Tracking around the accelerator ring
</center>

# Many macro-particle codes...

... with (transverse) collective effects for synchrotrons, e.g.:

- `PATRIC` (GSI)
- `MICROMAP` (GSI)
- `IMPACT` (Berkeley)
- `Synergia` (Fermilab)
- `(Py)ORBIT` (SNS)
- `SIMPSONS` (J-PARC)
- `elegant` (Argonne)
- `MAD-X SC` (CERN)
- `HEADTAIL`/`PyHEADTAIL` (CERN, GSI)
- ...

# 1. `PyHEADTAIL`: collective beam dynamics

# The `PyHEADTAIL` library

GPU-enabled, python based code for simulating collective beam dynamics in synchrotrons: [github repo $\nearrow$](https://github.com/PyCOMPLETE/PyHEADTAIL)

$\implies$ matrix-based transverse and non-linear longitudinal <span style="color:#00aad4;">tracking</span>

$\implies$ detailed models for <span style="color:#aa0000;">collective effect kicks</span>

<center>
<img src="images/wakefield.png" alt="wake field sketch" style="width:30%;"/>
Kick example: wakefield from leading particles affects trailing particles
</center>

# `PyHEADTAIL` on one slide

- origin: `C` code `HEADTAIL` by G. Rumolo (2000), 5 PR-STAB and 2 PRL papers
- modernised `PyHEADTAIL` used for:
    - **impedance-driven instabilities**: head-tail modes, transverse mode coupling instability (TMCI), coupled-bunch instabilities, ...
    - **space charge**
    - **electron cloud instabilities**
    - **mitigation techniques**: feedbacks, octupoles for Landau damping, multi-harmonic RF, ...

<center>
<div style='display: table; width: 30%;'>
    <div style='display: table-row; clear: both;'>
        <div style='float: left; display: table-cell; width: 50%;'>
            <img src='images/pyheadtail-instability.png' alt='head-tail instability' style='width: 90%;' />
        </div>
        <div style='float: left; display: table-cell; width: 50%;'>
            <img src='images/pyheadtail-tmci.png' alt='TMCI' style='width: 90%;' />
        </div>
    </div>
    <div style='display: table-row; clear: both;'>
        <div style='float: left; display: table-cell; width: 50%;'>
            <img src='images/pyheadtail-pinch.png' alt='e-cloud pinch' style='width: 90%;' />
        </div>
        <div style='float: left; display: table-cell; width: 50%;'>
            <img src='images/pyheadtail-hollow.png' alt='hollow bunching as mitigation' style='width: 90%;' />
        </div>
    </div>
</div>
</center>

# `PyHEADTAIL` modules

- wakefields:
    - resistive wall
    - broad-band resonator
    - custom wake tables
- space charge 
    - selfconsistent PIC
        - longitudinal, (slice-by-slice) transverse, 3D
        - open boundary Green's function (FFT)
        - open/rectangular boundary finite difference
    - frozen / adaptive field maps


- feedback (ideal and realistic [separate version])
- RF quadrupoles
- synchrotron radiation
- e-cloud pinch
- e-lens

# Active development at CERN and GSI

Contributions from $>13$ developers since 2014

<center>
    <img src="images/git-pyheadtail.png" alt="github activity on github repo" style="width:30%;" />
</center>

# Resources

- [PyHEADTAIL playground $\nearrow$](https://github.com/PyCOMPLETE/PyHEADTAIL-playground/): interactive tutorials
- [Overview of PyHEADTAIL $\nearrow$](https://cds.cern.ch/record/2672381/): Oeftiger, A. _An Overview of PyHEADTAIL._ CERN-ACC-NOTE-2019-0013.
- [PyHEADTAIL + SixTrackLib on GPU $\nearrow$](https://github.com/aoeftiger/pyhep2020): interactive notebook talk
- [SWAN beam dynamics gallery $\nearrow$](https://swan.web.cern.ch/swan/content/beam-dynamics.html): CERN platform to launch tutorials

<img src='images/swan-bd-gallery.png' alt='SWAN beam dynamics gallery' style='width: 10%; margin: auto;' />

# 2. Wakefields &amp; Dipole Moment Instabilities

# `PyHEADTAIL` wake model

<br />

<center>
    <img src="images/pyheadtail-wakemodel.png" alt="bunch and wakefield discretisation into slices" />
    <small><a href="https://www2.kek.jp/accl/seminar/file/PyHEADTAIL_PyECLOUD_2.pdf">courtesy Michael Schenk</a></small>
</center>

# Dipole wake fields (driving impedance)

Transverse kick on slice $i$ from previous slices due to wake $W_d$:

$$\Delta x'[i] \propto \sum\limits_{j=0}^i N[j] \quad \langle x \rangle [j] \quad W_d(z[i] - z[j])$$

- no synchrotron motion:
    - kicks accumulate, linac-like **beam breakup**

- with synchrotron motion: feedback loop
    - chroma $Q'=0$: 
        - synchrotron sidebands separate $\rightarrow$ **stable beam**
        - synchrotron sidebands couple $\rightarrow$ transverse mode coupling **instability**
    - chroma $Q'\neq 0$: **head-tail instability**

# Simulating head-tail instability in SIS100

$\rightarrow$ use `PyHEADTAIL` to simulate head-tail instability due to resistive wall impedance of SIS100 vacuum tube:

In [ ]:
# set up plotting etc
from imports import *

Importing PyHEADTAIL:

In [ ]:
import PyHEADTAIL

# Setting up <span style="color:#00aad4;">tracking</span> around the SIS100 ring

Transverse tracking:

In [ ]:
from PyHEADTAIL.trackers.transverse_tracking import TransverseMap
from PyHEADTAIL.trackers.detuners import Chromaticity

from pyheadtail_setup import transverse_map_kwargs, Q_x, Q_y
print ('chosen horizontal tune: {:.2f}, vertical tune: {:.2f}'.format(Q_x, Q_y))

Low, slightly positive chromaticity below transition $=$ strong growth rate of rigid bunch mode

In [ ]:
transverse_map = TransverseMap(
    detuners=[Chromaticity(
        Qp_x=5, Qp_y=5)],
    **transverse_map_kwargs,
)

Longitudinal tracking:

In [ ]:
from PyHEADTAIL.trackers.longitudinal_tracking import RFSystems

from pyheadtail_setup import longitudinal_map_kwargs

In [ ]:
longitudinal_map = RFSystems(**longitudinal_map_kwargs)

# Setting up <span style="color:#aa0000;">collective effect kick</span>

Simulate wakefields in bunch via circular resistive wall model:

In [ ]:
from PyHEADTAIL.impedances.wakes import WakeField, CircularResistiveWall
from PyHEADTAIL.particles.slicing import UniformBinSlicer

In [ ]:
# responsible for binning the beam longitudinally:
slicer = UniformBinSlicer(n_slices=40, n_sigma_z=3)

In [ ]:
resistive_wall_wake = CircularResistiveWall(
    pipe_radius=0.03,
    resistive_wall_length=longitudinal_map_kwargs['circumference'],
    conductivity=1.4e6,
    dt_min=None,
    n_turns_wake=1, 
)
wakefield = WakeField(slicer, resistive_wall_wake)

# Assembling the map around the SIS100 ring:

In [ ]:
pyht_ring_elements = list(transverse_map) + [longitudinal_map, wakefield]

<center>
        <img src='images/sis100.jpg' alt='SIS100 layout' style='width: 50%;' />
</center>

# Initialising the particle bunch

In [ ]:
from PyHEADTAIL.particles.generators import generate_Gaussian6DTwiss

from pyheadtail_setup import beam_kwargs

In [ ]:
n_macroparticles = 2000
intensity = 6.25e10

np.random.seed(0)

pyht_beam = generate_Gaussian6DTwiss(
    macroparticlenumber=n_macroparticles,
    intensity=intensity,
    **beam_kwargs,
    **transverse_map.get_injection_optics(
        for_particle_generation=True),
)

We'll give the bunch an initial kick:

In [ ]:
pyht_beam.y += 0.1 * pyht_beam.sigma_y()

In [ ]:
slices0 = pyht_beam.get_slices(slicer, statistics=['mean_y'])

resistive_wall_wake.dt_min = slices0.convert_to_time(slices0.slice_widths[0])

# Let's go – simulating SIS100 in `PyHEADTAIL`:

In [ ]:
n_turns = 10000

my = np.zeros(n_turns, dtype=float)

# loop over turns
for i in range(n_turns):
    # loop over elements around ring
    for element in pyht_ring_elements:
        element.track(pyht_beam)

    # record vertical bunch centroid amplitude
    my[i] = pyht_beam.mean_y()

# Outcome of our simulation?

In [ ]:
plt.plot(my * 1e3)
plt.xlabel('Turns')
plt.ylabel(r'$\langle y \rangle$ [mm]')
plt.title('Vertical bunch centroid motion');

$\leadsto$ centre-of-mass of the bunch grows exponentially $\implies$ **instability**!

Let's fit the exponential growth rate:

In [ ]:
growth_rate, _ = np.polyfit(np.arange(n_turns), np.log(my**2) / 2., 1)

print ('growth time: {:.0f} turns ~ {:.3f} sec'.format(
    1 / growth_rate, 1083.6 / (0.567 * 3e8) / growth_rate))

In [ ]:
plt.plot(my * 1e3)
plt.plot(np.arange(n_turns), 
         max(abs(my[:10]))* 0.8 * 1e3 * 
         np.exp(growth_rate * np.arange(n_turns)),
         ls='--')
plt.xlabel('Turns')
plt.ylabel(r'$\langle y \rangle$ [mm]')
plt.title('Vertical bunch centroid motion');

In [ ]:
from imports import plot_intrabunch

plot_intrabunch(pyht_ring_elements, pyht_beam, slicer, slices0)

$\implies$ indeed a rigid bunch mode, no nodes!

# Common mitigation strategies for head-tail instabilities

1. change **chromaticity**: sextupole magnets   
2. Landau damping: **octupole** magnets
3. linear **coupling**: skew quadrupole magnets
4. active transverse **feedback** (damper)

# Mitigation 1: Chromaticity

- higher-order modes $=$ lower growth rate
- for below (above) transition $\rightarrow$ go to negative (positive) chromaticity
- mode growth rates e.g. from Sacherer approach (1974)

<center>
    <img src='images/cps_modes_chroma.png' alt='CERN PS: mode growth rates vs. chromaticity' style='width: 50%;' />
    <small>example CERN PS <br /> taken from <a href="https://cds.cern.ch/record/279023/files/ps-95-002.pdf">R. Cappi, Observation of high-order head-tail instabilities at the CERN-PS, CERN/PS 95-02</a></small>
</center>

# Mitigation 2: Octupoles

- detuning with transverse amplitude $=$ tune spread $\leadsto$ Landau damping
- dispersion relations to solve for stability diagrams

<center>
    <img src="images/kornilov_SD.png" alt="SIS100 stability diagram for octupoles" />
    <small><a href="https://wiki.gsi.de/foswiki/pub/SIS100BD/MacTalksBD/Kornilov_MAC20b.pdf">V. Kornilov on MAC20b</a> or PRSTAB 11, 014201 (2008)</small>
</center>

# Mitigation 3: Coupling

- mix growth rates of transverse planes (e.g. CERN PS)
- "share" Landau damping

See e.g. [Métral, E. Theory of coupled Landau damping. Part. Accel., 62 (1999) 259. $\nearrow$]( https://cds.cern.ch/record/380927/files/p259.pdf)

# Mitigation 4: Damper

Let's use the [foreseen SIS100 damper with $100$ turns $\nearrow$](https://wiki.gsi.de/foswiki/pub/SIS100BD/MacTalksBD/Kornilov_MAC20b.pdf) to stabilise the previously simulated head-tail instability:

In [ ]:
from PyHEADTAIL.feedback.transverse_damper import TransverseDamper

damper = TransverseDamper(dampingrate_x=100, dampingrate_y=100)

In [ ]:
pyht_ring_elements += [damper]

### Continue simulation with damper on now:

In [ ]:
n_turns_2 = 1000

my_2 = np.zeros(n_turns_2, dtype=float)

# loop over turns
for i in range(n_turns_2):
    # loop over elements around ring
    for element in pyht_ring_elements:
        element.track(pyht_beam)

    # record vertical bunch centroid amplitude
    my_2[i] = pyht_beam.mean_y()

# Outcome of our follow-up simulation?

In [ ]:
plt.plot(np.concatenate((my, my_2)) * 1e3)
plt.axvline(n_turns, color='black', ls='--')
plt.text(n_turns, plt.ylim()[1] * 0.9, r'damper on $\rightarrow$ ',
         fontsize=20, ha='right', va='top', bbox={'facecolor': 'white', 'alpha': 0.5})
plt.xlabel('Turns')
plt.ylabel(r'$\langle y \rangle$ [mm]')
plt.title('Vertical bunch centroid motion');

$\implies$ stabilised beam!

$\leadsto$ important: these `PyHEADTAIL` simulations are kept simple to be instructive, same model can be extended and used in higher resolution for proper study!

# 3. `SixTrackLib`: symplectic non-linear 6D tracking

GPU-enabled C templated code with Python API for simulating single-particle beam dynamics: [github repo $\nearrow$](https://github.com/sixtrack/sixtracklib)

$\implies$ advanced non-linear thin-lens <span style="color:#00aad4;">tracking</span>, no further approximations

$\implies$ approximative / simplified models for <span style="color:#aa0000;">collective effect kicks</span>:
- fixed frozen space charge
- 4D and 6D beam-beam


<center>
<img src="images/tracking.png" alt="tracking sketch" style="width:30%;"/>
Example for tracking: <span style="color:#ff6701;">dipoles</span> and <span style="color:#800000;">(de-)</span><span style="color:#000080;">focusing quadrupoles</span>
</center>

<img src="images/stl_pyht_workflow.png" alt="SixTrackLib and PyHT workflow" style="width: 80%; margin: auto;" />

In [ ]:
import pysixtrack
import sixtracklib as stl

from cpymad.madx import Madx

## MAD-X part first

In [ ]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False
madx.options.info = False

Define SIS100 lattice in MAD-X:

In [ ]:
madx.call('./SIS100.seq')

Define SIS100 U${}^{28+}$ beam in MAD-X:

In [ ]:
from sixtracklib_setup import *

print ('mass = {:d}amu, charge = {:d}e, Ekin = {:.0f}MeV/u'.format(
    A, Q, 1e-6 * Ekin_per_nucleon))

In [ ]:
madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot);

In [ ]:
madx.input('''
            kqd := -0.2798446835;
            kqf := 0.2809756135;

            K1NL_S00QD1D :=  kqd ;
            K1NL_S00QD1F :=  kqf ;
            K1NL_S00QD2F :=  kqf ;
            K1NL_S52QD11 :=  1.0139780 * kqd ;
            K1NL_S52QD12 :=  1.0384325 * kqf ;
        ''');

In [ ]:
madx.use(sequence='sis100ring')

assert madx.command.select(
    flag='MAKETHIN',
    class_='QUADRUPOLE',
    slice_='9',
)

assert madx.command.select(
    flag='MAKETHIN',
    class_='SBEND',
    slice_='9',
)

assert madx.command.makethin(
    makedipedge=True,
    style='teapot',
    sequence='SIS100RING',
)

Match tunes to $Q_x=18.84$, $Q_y=18.73$:

In [ ]:
madx.use(sequence='sis100ring')

madx.input('''
match, sequence=SIS100RING;
global, sequence=SIS100RING, q1=18.84, q2=18.73;
vary, name=kqf, step=0.00001;
vary, name=kqd, step=0.00001;
lmdif, calls=500, tolerance=1.0e-10;
endmatch;
''');

In [ ]:
twiss = madx.twiss();

print ('\nQ1 = {:.2f} and Q2 = {:.2f}'.format(
    twiss.summary['q1'], twiss.summary['q2']))

## Now transfer SIS100 lattice to `PySixTrack`

In [ ]:
pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.sis100ring, 
    exact_drift=True, install_apertures=False,
)

`pysixtrack_elements.elements` is a python list, we can play with it...

## Finally go to `SixTrackLib`

Transfer lattice from `PySixTrack`:

In [ ]:
elements = stl.Elements.from_line(pysixtrack_elements)

nturns = 2**16
elements.BeamMonitor(num_stores=nturns);

And define particle set:

In [ ]:
npart = 1
particles = stl.Particles.from_ref(npart, p0c=p0c)

In [ ]:
particles.x += 1e-6
particles.y += 1e-6

The trackjob will define the simulation kernel for us:

In [ ]:
job = stl.TrackJob(elements, particles, device=None)

$\leadsto$ note the `device` argument:

this can be `'opencl:0.0'` for openCL parallelisation, e.g. for multi-core CPU or GPU!

### Let's track to confirm the tunes from `MAD-X`:

In [ ]:
job.track_until(nturns)
job.collect()

# Evaluation:

In [ ]:
rec_x = job.output.particles[0].x
rec_y = job.output.particles[0].y

In [ ]:
plt.plot(rec_x[:128])
plt.plot(rec_y[:128])

In [ ]:
from sixtracklib_setup import plot_tunes_from_stl_results

plot_tunes_from_stl_results(twiss, job)

$\implies$ tunes are on top of MAD-X, perfect!

# 4. Space Charge &amp; Resonances and 2nd Order Coherent Instabilities

`SixTrackLib` provides extremely fast simulations!

Including in the SIS100 model
- full lattice (from [gitlab repo $\nearrow$](https://git.gsi.de/sys/lattices/-/tree/master/SIS100/MADX))
- non-linear, frozen 3D space charge
- field error models until 7th order for dipole and quadrupole magnets
    
$~\longrightarrow~$ scan 2D tune diagrams, $\approx 2$min per simulation for 20'000 turns 

(on `virgo` cluster: 64 CPU cores or NVIDIA V100 GPU)

# Non-linear magnet resonances + space charge

$~\Longrightarrow~$ study space charge limit in SIS100 for heavy-ion cycles:

<center>
    <img src="images/sis100_lossdiagram_h1.png" alt="beam loss in tune diagram for SIS100" />
    <small>beam loss tune diagram for SIS100</small>
</center>

# Mitigation techniques

1. correcting beta-beat
2. double harmonic RF for bunch flattening
3. electron lenses (to be studied)

# Mitigation 1: Correcting Beta-beat

No gradient errors in the quadrupole magnets (but all other non-linear field errors):

<br />

<center>
    <img src="images/sis100_lossdiagram_h1_nobetabeat.png" alt="beta-beat corrected beam loss tune diagram for SIS100" />
    <small>beam loss tune diagram for SIS100</small>
</center>

# Mitigation 2: Double Harmonic RF

Double harmonic $\implies$ flattened bunch profile $\implies$ weaker space charge $\implies$ more space in tune diagram

<br />

<center>
<div style="width: 100%;">
    <div style="float: left; width: 50%;">
        <img src="images/sis100_lossdiagram_h1.png" alt="single-harmonic beam loss in tune diagram for SIS100" />
        <small>single harmonic</small>
    </div>
    <div style="float: left; width: 50%;">
        <img src="images/sis100_lossdiagram_h2.png" alt="double-harmonic beam loss in tune diagram for SIS100" />
        <small>double harmonic</small>
    </div>
</div>
</center>

# Compare space charge resonance behaviour

With the new merger `SixHead` $=$ `SixTrackLib` + `PyHEADTAIL`:

$\implies$ compare frozen space charge models with self-consistent 3D particle-in-cell space charge:

<br />

<center>
    <img src="images/induced-res.png" alt="space charge model comparison" style="max-width:80%;" />
    <small>1D tune scan across resonance</small>
</center>

# 2nd order (and higher) coherent instabilities

Investigating 90deg stop-band for higher-order coherent instabilities in synchrotrons with `SixHead` (10'000 turns on GPU with 4 million macro-particles in 4h):
    
<br />

<center>
    <img src="images/3d-90deg.png" alt="3D study on 90deg stop-band" style="max-width:80%;" />
    <small>1D tune scan across resonance</small>
</center>

# ... so long and thanks for all the fish ...

# Summary

New suite `SixHead` consisting of:
- symplectic 6D non-linear tracking with `SixTrackLib`
- collective beam dynamics with `PyHEADTAIL` (wakefields, space charge, feedbacks)
    
is parallelised on:
- multi-core CPU tracking
- GPU

and renders possible:
- accurate, fast and flexible simulation studies for SIS18 and SIS100 for high-intensity operation

# THANK YOU!